# Graph Classification

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader

import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd

import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

/home/boot/anaconda3/envs/zeyu1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max

from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from dataclasses import dataclass
from typing import Optional

import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor

@dataclass(init=False)
class SelectOutput:
    r"""The output of the :class:`Select` method, which holds an assignment
    from selected nodes to their respective cluster(s).

    Args:
        node_index (torch.Tensor): The indices of the selected nodes.
        num_nodes (int): The number of nodes.
        cluster_index (torch.Tensor): The indices of the clusters each node in
            :obj:`node_index` is assigned to.
        num_clusters (int): The number of clusters.
        weight (torch.Tensor, optional): A weight vector, denoting the strength
            of the assignment of a node to its cluster. (default: :obj:`None`)
    """
    node_index: Tensor
    num_nodes: int
    cluster_index: Tensor
    num_clusters: int
    weight: Optional[Tensor] = None

    def __init__(
        self,
        node_index: Tensor,
        num_nodes: int,
        cluster_index: Tensor,
        num_clusters: int,
        weight: Optional[Tensor] = None,
    ):
        if node_index.dim() != 1:
            raise ValueError(f"Expected 'node_index' to be one-dimensional "
                             f"(got {node_index.dim()} dimensions)")

        if cluster_index.dim() != 1:
            raise ValueError(f"Expected 'cluster_index' to be one-dimensional "
                             f"(got {cluster_index.dim()} dimensions)")

        if node_index.numel() != cluster_index.numel():
            raise ValueError(f"Expected 'node_index' and 'cluster_index' to "
                             f"hold the same number of values (got "
                             f"{node_index.numel()} and "
                             f"{cluster_index.numel()} values)")

        if weight is not None and weight.dim() != 1:
            raise ValueError(f"Expected 'weight' vector to be one-dimensional "
                             f"(got {weight.dim()} dimensions)")

        if weight is not None and weight.numel() != node_index.numel():
            raise ValueError(f"Expected 'weight' to hold {node_index.numel()} "
                             f"values (got {weight.numel()} values)")

        self.node_index = node_index
        self.num_nodes = num_nodes
        self.cluster_index = cluster_index
        self.num_clusters = num_clusters
        self.weight = weight




class Select(torch.nn.Module):
    r"""An abstract base class for implementing custom node selections as
    described in the `"Understanding Pooling in Graph Neural Networks"
    <https://arxiv.org/abs/1905.05178>`_ paper, which maps the nodes of an
    input graph to supernodes in the coarsened graph.

    Specifically, :class:`Select` returns a :class:`SelectOutput` output, which
    holds a (sparse) mapping :math:`\mathbf{C} \in {[0, 1]}^{N \times C}` that
    assigns selected nodes to one or more of :math:`C` super nodes.
    """
    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        pass

    def forward(self, *args, **kwargs) -> SelectOutput:
        raise NotImplementedError

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}()'

def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.

    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)

    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])

    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)

    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))

    return out

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i

    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]

    if edge_attr is not None:
        edge_attr = edge_attr[mask]

    return torch.stack([row, col], dim=0), edge_attr

def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        # Make sure that we do not drop all nodes in a graph.
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)

        perm = (x > scores_min).nonzero().view(-1)
        return perm

    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')

        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)

        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)

        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]

        return x_perm[batch_perm[mask]]

    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")

class Discriminator(torch.nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(in_channels * 2, in_channels)
        self.fc2 = nn.Linear(in_channels, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.sigmoid(self.fc2(x))
        return x


class CGIPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, non_lin=torch.tanh):
        super(CGIPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.non_lin = non_lin
        self.hidden_dim = in_channels
        self.transform = GraphConv(in_channels, self.hidden_dim)
        self.pp_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.np_conv = GraphConv(self.hidden_dim, self.hidden_dim)

        self.positive_pooling = GraphConv(self.hidden_dim, 1)
        self.negative_pooling = GraphConv(self.hidden_dim, 1)

        self.discriminator = Discriminator(self.hidden_dim)
        self.loss_fn = torch.nn.BCELoss()

    def forward(self, x, edge_index, edge_attr=None, batch=None):
        device = x.device  # 获取输入张量的设备信息

        if batch is None:
            batch = edge_index.new_zeros(x.size(0))

        x_transform = F.leaky_relu(self.transform(x, edge_index), 0.2)
        x_tp = F.leaky_relu(self.pp_conv(x, edge_index), 0.2)
        x_tn = F.leaky_relu(self.np_conv(x, edge_index), 0.2)
        s_pp = self.positive_pooling(x_tp, edge_index).squeeze()
        s_np = self.negative_pooling(x_tn, edge_index).squeeze()

        perm_positive = topk(s_pp, 1, batch)
        perm_negative = topk(s_np, 1, batch)
        x_pp = x_transform[perm_positive] * self.non_lin(s_pp[perm_positive]).view(-1, 1)
        x_np = x_transform[perm_negative] * self.non_lin(s_np[perm_negative]).view(-1, 1)

        x_pp_readout = gap(x_pp, batch[perm_positive])
        x_np_readout = gap(x_np, batch[perm_negative])
        x_readout = gap(x_transform, batch)

        positive_pair = torch.cat([x_pp_readout, x_readout], dim=1)
        negative_pair = torch.cat([x_np_readout, x_readout], dim=1)

        real = torch.ones(positive_pair.shape[0], device=device)  # 将张量移动到相应设备
        fake = torch.zeros(negative_pair.shape[0], device=device)  # 将张量移动到相应设备
        #real_loss = self.loss_fn(self.discriminator(positive_pair), real)
        #fake_loss = self.loss_fn(self.discriminator(negative_pair), fake)
        #discrimination_loss = (real_loss + fake_loss) / 2

        score = (s_pp - s_np)

        perm = topk(score, self.ratio, batch)
        x = x_transform[perm] * self.non_lin(score[perm]).view(-1, 1)
        batch = batch[perm]

        filter_edge_index, filter_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, filter_edge_index, filter_edge_attr, batch, perm

### MUTAG

In [6]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/Zeyu/Pooling/1"

dataset_sparse = TUDataset(root=data_path, name="MUTAG", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 12.02 seconds
Var Time: 0.01 seconds
Average Memory: 48.67 MB
Average Best Val Acc: 0.8690
Std Best Test Acc: 0.0488
Average Test Acc: 0.8621


### DD

In [8]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="DD", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 31.07 seconds
Var Time: 0.17 seconds
Average Memory: 1636.00 MB
Average Best Val Acc: 0.7508
Std Best Test Acc: 0.0147
Average Test Acc: 0.7207


### IMDB-BINARY

In [4]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="IMDB-BINARY", transform=T.Compose([T.OneHotDegree(136)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.7)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.7)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6974, Val Acc: 0.4800, Test Acc: 0.5000
Seed: 42, Epoch: 002, Loss: 0.6918, Val Acc: 0.5533, Test Acc: 0.5667
Seed: 42, Epoch: 003, Loss: 0.6868, Val Acc: 0.6333, Test Acc: 0.6467
Seed: 42, Epoch: 004, Loss: 0.6797, Val Acc: 0.7067, Test Acc: 0.7067
Seed: 42, Epoch: 005, Loss: 0.6765, Val Acc: 0.7000, Test Acc: 0.7867
Seed: 42, Epoch: 006, Loss: 0.6685, Val Acc: 0.6933, Test Acc: 0.7867
Seed: 42, Epoch: 007, Loss: 0.6585, Val Acc: 0.7067, Test Acc: 0.7667
Seed: 42, Epoch: 008, Loss: 0.6525, Val Acc: 0.7067, Test Acc: 0.7733
Seed: 42, Epoch: 009, Loss: 0.6398, Val Acc: 0.7067, Test Acc: 0.7533
Seed: 42, Epoch: 010, Loss: 0.6223, Val Acc: 0.7000, Test Acc: 0.7733
Seed: 42, Epoch: 011, Loss: 0.6214, Val Acc: 0.7067, Test Acc: 0.7733
Seed: 42, Epoch: 012, Loss: 0.6033, Val Acc: 0.7000, Test Acc: 0.7733
Seed: 42, Epoch: 013, Loss: 0.5968, Val Acc: 0.7067, Test Acc: 0.8000
Seed: 42, Epoch: 014, Loss: 0.5936, Val Acc: 0.7467, Test Acc: 0.7933
Seed: 42, Epoch: 015

### IMDB-MULTI

In [10]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="IMDB-MULTI", transform=T.Compose([T.OneHotDegree(88)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 189 for seed 44
Average Time: 49.68 seconds
Var Time: 1.83 seconds
Average Memory: 328.67 MB
Average Best Val Acc: 0.5259
Std Best Test Acc: 0.0202
Average Test Acc: 0.4622


### COLLAB

In [11]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="COLLAB", transform=T.Compose([T.OneHotDegree(491)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_CGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = CGIPool(hidden_channels, ratio=0.7)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = CGIPool(hidden_channels, ratio=0.7)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, perm = self.pool1(x, edge_index, None, batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, perm = self.pool2(x, edge_index, None, batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_CGI(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.0794, Val Acc: 0.3493, Test Acc: 0.3027
Seed: 42, Epoch: 002, Loss: 0.9672, Val Acc: 0.4133, Test Acc: 0.3813
Seed: 42, Epoch: 003, Loss: 0.9169, Val Acc: 0.5213, Test Acc: 0.4693
Seed: 42, Epoch: 004, Loss: 0.8609, Val Acc: 0.6987, Test Acc: 0.6427
Seed: 42, Epoch: 005, Loss: 0.8011, Val Acc: 0.7093, Test Acc: 0.6760
Seed: 42, Epoch: 006, Loss: 0.7357, Val Acc: 0.6840, Test Acc: 0.6627
Seed: 42, Epoch: 007, Loss: 0.6806, Val Acc: 0.6747, Test Acc: 0.6613
Seed: 42, Epoch: 008, Loss: 0.6358, Val Acc: 0.6613, Test Acc: 0.6573
Seed: 42, Epoch: 009, Loss: 0.6139, Val Acc: 0.6720, Test Acc: 0.6787
Seed: 42, Epoch: 010, Loss: 0.5901, Val Acc: 0.6720, Test Acc: 0.6760
Seed: 42, Epoch: 011, Loss: 0.5742, Val Acc: 0.6747, Test Acc: 0.6813
Seed: 42, Epoch: 012, Loss: 0.5639, Val Acc: 0.7133, Test Acc: 0.6960
Seed: 42, Epoch: 013, Loss: 0.5534, Val Acc: 0.7147, Test Acc: 0.6987
Seed: 42, Epoch: 014, Loss: 0.5393, Val Acc: 0.7160, Test Acc: 0.6960
Seed: 42, Epoch: 015

# Graph Regression

### QM7

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Existed dataset loaded: datasets/processed/qm7.pt

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500MAE=1549.9229 MAE=1547.7614 MAE=1544.4138 MAE=1542.0225 MAE=1538.9407 MAE=1535.5701 MAE=1532.5792 MAE=1528.6232 MAE=1523.9182 Epoch: 10/500MAE=1519.6655 MAE=1513.7695 MAE=1509.2549 MAE=1503.9119 MAE=1497.4292 MAE=1491.0306 MAE=1483.8711 MAE=1477.2434 MAE=1470.3829 MAE=1460.3391 Epoch: 20/500MAE=1456.4886 MAE=1447.4862 MAE=1433.5365 MAE=1420.7318 MAE=1406.0607 MAE=1399.5156 MAE=1385.3484 MAE=1376.6484 MAE=1351.6215 MAE=1362.4369 Epoch: 30/500MAE=1347.3710 MAE=1328.2316 MAE=1316.1289 MAE=1302.1105 MAE=1291.4336 MAE=1271.5015 MAE=1246.5602 MAE=1254.4756 MAE=1222.1191 MAE=1211.5674 Epoch: 40/500MAE=1192.2863 MAE=1176.5430 MAE=1161.6340 MAE=1142.7424 MAE=1125.0427 MAE=1084.7528 MAE=1068.3932 MAE=1061.0107 MAE=1030.2260 MAE=1019.5750 Epoch: 50/500MAE=1008.9548 MAE=989.5651 MAE=952.6211 MAE

### QM8

In [2]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num 0 --run_times=5 --patience=10 --epochs=150 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num 0 --run_times=5 --patience=10 --epochs=150 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num 0 --run_times=5 --patience=10 --epochs=150 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num 0 --run_times=5 --patience=10 --epochs=150 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num 0 --run_times=5 --patience=10 --epochs=150 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Existed dataset loaded: datasets/processed/qm8.pt

Current dataset: qm8, include 21783 molecules and 12 regression tasks

Splitting, finish 1/1  
Epoch: 1/150MAE=0.1306 MAE=0.0994 MAE=0.0766 MAE=0.0557 MAE=0.0419 MAE=0.0346 MAE=0.0345 MAE=0.0331 MAE=0.0376 Epoch: 10/150MAE=0.0352 MAE=0.0326 MAE=0.0297 MAE=0.0295 MAE=0.0282 MAE=0.0280 MAE=0.0276 MAE=0.0290 MAE=0.0291 MAE=0.0326 Epoch: 20/150MAE=0.0294 MAE=0.0271 MAE=0.0282 MAE=0.0266 MAE=0.0263 MAE=0.0270 MAE=0.0263 MAE=0.0265 MAE=0.0255 MAE=0.0252 Epoch: 30/150MAE=0.0257 MAE=0.0250 MAE=0.0253 MAE=0.0250 MAE=0.0249 MAE=0.0259 MAE=0.0255 MAE=0.0250 MAE=0.0261 MAE=0.0251 Epoch: 40/150MAE=0.0246 MAE=0.0250 MAE=0.0245 MAE=0.0248 MAE=0.0251 MAE=0.0245 MAE=0.0247 MAE=0.0248 MAE=0.0246 MAE=0.0244 Epoch: 50/150MAE=0.0245 MAE=0.0247 MAE=0.0246 MAE=0.0245 MAE=0.0244 MAE=0.0245 MAE=0.0245 MAE=0.0244 MAE=0.0244 MAE=0.0244 Epoch: 60/150MAE=0.0244 MAE=0.0243 MAE=0.0243 MAE=0.0243 MAE=0.0242 MAE=0.0243

### BACE

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++


Existed dataset loaded: datasets/processed/bace.pt

Current dataset: bace, include 1513 molecules and 1 classification tasks

Splitting, finish 1/1  
Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 1's run over********************
AUROC: 0.812 +/- 0.000
AUPRC: 0.758 +/- 0.000

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 2's run over********************
AUROC: 0.830 +/- 0.018
AUPRC: 0.788 +/- 0.030

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 3's run over********************
AUROC: 0.826 +/- 0.016
AUPRC: 0.780 +/- 0.027

Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 4's run over********************
AUROC: 0.829 +/- 0.015
AUPRC: 0.789 +/- 0.028

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150Epoch: 40/150Epoch: 50/150
********************1's fold 5's run over********************
AUROC: 0.820 +/- 0.022
AUPRC: 0.777 +/- 0.035

++++++++++++++++++++++0.3+++++++

### ESOL

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Existed dataset loaded: datasets/processed/esol.pt

Current dataset: esol, include 1127 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=2.0718 RMSE=1.8997 RMSE=1.7036 RMSE=1.6062 RMSE=2.2478 RMSE=1.6341 RMSE=2.5753 RMSE=2.0837 RMSE=3.1815 Epoch: 10/150RMSE=1.5465 RMSE=1.3421 RMSE=1.6950 RMSE=1.9844 RMSE=1.2863 RMSE=1.4566 RMSE=1.2869 RMSE=1.3106 RMSE=1.3919 RMSE=2.1109 Epoch: 20/150RMSE=2.2135 RMSE=2.1410 RMSE=2.2133 RMSE=2.1879 RMSE=2.2280 RMSE=1.2859 RMSE=1.3158 RMSE=2.2738 RMSE=2.2767 RMSE=2.2453 Epoch: 30/150RMSE=2.2386 RMSE=1.4225 RMSE=1.3738 RMSE=1.4043 RMSE=1.3963 RMSE=1.4424 RMSE=1.4113 RMSE=1.2289 RMSE=1.4463 RMSE=1.2472 Epoch: 40/150RMSE=1.2314 RMSE=1.2365 RMSE=1.2601 RMSE=1.2488 RMSE=1.2393 RMSE=1.2374 RMSE=1.2354 RMSE=1.2351 RMSE=1.2408 RMSE=1.2493 Epoch: 50/150RMSE=1.2474 RMSE=1.2419 RMSE=1.2380 RMSE=1.2424 RMSE=1.2471 RMSE=1.2397 RMSE=1.2384 RMSE=2.4182 
********************1's fold 1's run over*

### Freesolv

In [4]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++


Existed dataset loaded: datasets/processed/freesolv.pt

Current dataset: freesolv, include 639 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=5.3008 RMSE=4.9119 RMSE=3.8208 RMSE=2.5792 RMSE=2.5959 RMSE=2.4530 RMSE=2.2121 RMSE=2.1744 RMSE=2.1863 Epoch: 10/150RMSE=2.0348 RMSE=2.3669 RMSE=2.1571 RMSE=2.2296 RMSE=2.0895 RMSE=2.1718 RMSE=2.2484 RMSE=2.1528 RMSE=2.2124 RMSE=2.2672 Epoch: 20/150RMSE=2.2282 RMSE=2.2321 RMSE=2.2446 RMSE=2.2236 RMSE=2.1834 RMSE=2.2173 RMSE=2.1675 RMSE=2.1655 RMSE=2.1718 RMSE=2.1673 Epoch: 30/150RMSE=2.1664 RMSE=2.2518 
********************1's fold 1's run over********************
RMSE: 2.252 +/- 0.000

Epoch: 1/150RMSE=5.1971 RMSE=4.7546 RMSE=3.5074 RMSE=2.5835 RMSE=2.3011 RMSE=2.1394 RMSE=2.0396 RMSE=1.9223 RMSE=2.0062 Epoch: 10/150RMSE=1.9697 RMSE=1.9506 RMSE=1.9269 RMSE=1.8852 RMSE=1.7383 RMSE=1.7836 RMSE=1.8263 RMSE=1.7406 RMSE=1.8362 RMSE=1.7515 Epoch: 20/150RMSE=1.7326 RMSE=1.7348 RMSE=1.7668 RMSE=1.7659 RMSE=1.7968 RMSE=1.7990 

### Lipophilicity

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.1 --pooling='CGI'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.3 --pooling='CGI'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.5 --pooling='CGI'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.7 --pooling='CGI'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --cgi_ratio=0.9 --pooling='CGI'

++++++++++++++++++++++0.1++++++++++++++++++++++++


Existed dataset loaded: datasets/processed/lipo.pt

Current dataset: lipo, include 4200 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=1.4377 RMSE=1.2825 RMSE=1.3415 RMSE=1.3214 RMSE=1.3303 RMSE=1.2929 RMSE=1.3297 RMSE=1.2325 RMSE=1.2157 Epoch: 10/150RMSE=1.2426 RMSE=1.3559 RMSE=1.3393 RMSE=1.2666 RMSE=1.2662 RMSE=1.2719 RMSE=1.2364 RMSE=1.2612 RMSE=1.3486 RMSE=1.3924 Epoch: 20/150RMSE=1.2633 RMSE=1.2863 RMSE=1.2886 RMSE=1.1201 RMSE=1.1211 RMSE=1.1271 RMSE=1.1190 RMSE=1.2668 RMSE=1.2434 RMSE=1.1201 Epoch: 30/150RMSE=1.1192 RMSE=1.1222 RMSE=1.1246 RMSE=1.1250 RMSE=1.1269 RMSE=1.1188 RMSE=1.3982 RMSE=1.3945 RMSE=1.3867 RMSE=1.3915 Epoch: 40/150RMSE=1.3925 RMSE=1.3952 RMSE=1.3952 RMSE=1.3949 RMSE=1.3946 RMSE=1.3951 RMSE=1.3970 RMSE=1.1440 
********************1's fold 1's run over********************
RMSE: 1.144 +/- 0.000

Epoch: 1/150RMSE=1.3863 RMSE=1.2456 RMSE=1.2249 RMSE=1.1782 RMSE=1.3677 RMSE=1.4398 RMSE=1.3274 RMSE=1.3988 RMSE=1.3707 Epoch: 10/150RMSE=1.